In [1]:
import numpy as np
import pandas as pd
from pprint import pprint
import yfinance as yf
from scipy.optimize import minimize
import random

In [2]:
tickers=["AAPL","MSFT","AMZN","GOOGL"]
num_stocks=len(tickers)

cl_price=pd.DataFrame()

In [3]:
for ticker in tickers:
    cl_price[ticker]=yf.download(ticker,period="1y",interval="1d")["Adj Close"]

cl_price.to_csv("Closing_price.csv")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
returns = cl_price.pct_change().dropna()
expected_returns = returns.mean()*252
covariance_matrix = returns.cov()*252

def portfolio_sharpe_ratio(weights):
    portfolio_return = np.sum(expected_returns * weights)
    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    sharpe_ratio = portfolio_return / portfolio_risk
    return -sharpe_ratio 

def create_random_portfolio():
    weights = np.array([random.uniform(0, 1) for _ in range(num_stocks)])
    return weights / sum(weights) 

population_size = 100
generations = 50
population = [create_random_portfolio() for _ in range(population_size)]

for _ in range(generations):
    fitness_values = [portfolio_sharpe_ratio(weights) for weights in population]
    elite_indices = np.argsort(fitness_values)[:population_size // 2]
    elite_population = [population[i] for i in elite_indices]
    new_population = []
    for _ in range(population_size - len(elite_population)):
        parent1, parent2 = random.choices(elite_population, k=2)
        alpha = random.uniform(0.2, 0.8)
        child = alpha * parent1 + (1 - alpha) * parent2
        mutation_rate = 0.1
        if random.random() < mutation_rate:
            mutation = np.random.normal(0, 0.05, num_stocks)  # Add a small amount of noise
            child += mutation
        child = np.maximum(0, child)
        child = child / sum(child) 
        new_population.append(child)
    population = elite_population + new_population    # Replace the old population with the new generation

best_portfolio_weights = max(population, key=portfolio_sharpe_ratio)


print("Optimal Asset Allocations using Genetic Algorithm:")
for i, symbol in enumerate(tickers):
    print(f"{symbol}: {best_portfolio_weights[i]:.4f}")

best_portfolio_return = np.sum(expected_returns * best_portfolio_weights)
best_portfolio_risk = np.sqrt(np.dot(best_portfolio_weights.T, np.dot(covariance_matrix, best_portfolio_weights)))
sharpe_ratio= best_portfolio_return/best_portfolio_risk


print(f"Expected Portfolio Return: {best_portfolio_return:.4f}")
print(f"Portfolio Risk (Standard Deviation): {best_portfolio_risk:.4f}")
print(f"Sharpe Ratio : {sharpe_ratio:.4f}")

Optimal Asset Allocations using Genetic Algorithm:
AAPL: 0.4081
MSFT: 0.2566
AMZN: 0.2524
GOOGL: 0.0829
Expected Portfolio Return: 0.5021
Portfolio Risk (Standard Deviation): 0.2100
Sharpe Ratio : 2.3917
